# Subsequence Shenanigans

## Problem Statement
Given lower and upper bounds and a list of values, count the number of sub-sequences which have a maximum value within those bounds, including overlapping sub-sub-sequences.  Bonus points for deriving this count in linear time (O(n)) and constant space.

For example: given bounds of `[2,3]` and the list `[2,1,4,3]`, there are **3** valid sub-sequences: `[2], [2,1], [3]`

(I have no idea why the count alone is useful, and not the sub-sequences themselves, but let's roll with it.)

## Finding Valid Sequences
Generalizing the problem a bit, items can be split into three categories:
- **disallowed:** cannot appear in a valid run of items; value > upper bound
- **required:** must appear for a run to be valid; lower bound <= value <= upper bound
- **optional:** may appear in a valid run, but not itself sufficient; value < lower bound

In [1]:
lower_bound = 2
upper_bound = 3
seq = [2,1,4,3]

def is_disallowed(i):
    return i > upper_bound
def is_required(i):
    return lower_bound <= i <= upper_bound
def is_optional(i):
    return i < lower_bound

def find_valid_runs(seq):
    valid_runs = []
    current_run = []
    run_qualifies = False
    for i in seq:
        if is_disallowed(i):
            if run_qualifies:
                valid_runs.append(current_run)
                current_run = []
                run_qualifies = False
            continue
        run_qualifies = run_qualifies or is_required(i)
        current_run.append(i)
    if run_qualifies:
        valid_runs.append(current_run)

    return valid_runs

print(find_valid_runs(seq))

[[2, 1], [3]]


## Those Pesky Sub-Sequences
At the top level, we can now find valid runs of values, but only the longest non-overlapping ones; what about `[2]`?  Or, if there are longer runs that have valid sub-sequences, how do we find them?

A few observations:
- If all items in a run are `required` then all sub-sequences must be valid (if all items are valid and sufficient, combinations must be valid and sufficient; we'll figure out `optional` items later)
- When we add another item to a run, all previous valid sub-sequences must still apply
- *And* all new valid sub-sequences must use that new item (it is the only new thing)

Thus:

In [17]:
# For most of this, we're going to use these bounds as I think they make example sequences easier to understand
lower_bound = 2
upper_bound = 8

def trailing_subs(run):
    return [ run[i:] for i in range(-len(run), 0) ]

def series_of_runs(seq):
    run = []
    for i in seq:
        run.append(i)  # We're just going to assume `i` is a "required" element for this
        new_subs = trailing_subs(run)
        print(f"{len(run) = }: {new_subs}  {len(new_subs) = }")

seq = [2,3,4,5]
series_of_runs(seq)

len(run) = 1: [[2]]  len(new_subs) = 1
len(run) = 2: [[2, 3], [3]]  len(new_subs) = 2
len(run) = 3: [[2, 3, 4], [3, 4], [4]]  len(new_subs) = 3
len(run) = 4: [[2, 3, 4, 5], [3, 4, 5], [4, 5], [5]]  len(new_subs) = 4


Nifty!  The number of new subsequences is the same as the number of items in the run!  (*If* all items are in the `required` category)

## OK, So What About Optional Items?
Hmm... what happens when we start adding `optional` items?  Here's two `required` items and some optional ones:

In [14]:
seq = [2,3,1,1,1]
series_of_runs(seq)

len(run) = 1: [[2]]  len(new_subs) = 1
len(run) = 2: [[2, 3], [3]]  len(new_subs) = 2
len(run) = 3: [[2, 3, 1], [3, 1], [1]]  len(new_subs) = 3
len(run) = 4: [[2, 3, 1, 1], [3, 1, 1], [1, 1], [1]]  len(new_subs) = 4
len(run) = 5: [[2, 3, 1, 1, 1], [3, 1, 1, 1], [1, 1, 1], [1, 1], [1]]  len(new_subs) = 5


After the second item, adding `optional` ones creates some new valid sequences, but others aren't:

> len(run) = 1: [[2]]  len(new_subs) = 1  
> len(run) = 2: [[2, 3], [3]]  len(new_subs) = 2  
> len(run) = 3: [[2, 3, 1], [3, 1], ~~[1]~~]  len(new_subs) = ~~3~~ **2**  
> len(run) = 4: [[2, 3, 1, 1], [3, 1, 1], ~~[1, 1]~~, ~~[1]~~]  len(new_subs) = ~~4~~ **2**  
> len(run) = 5: [[2, 3, 1, 1, 1], [3, 1, 1, 1], ~~[1, 1, 1]~~, ~~[1, 1]~~, ~~[1]~~]  len(new_subs) = ~~5~~ **2**  

The number of new valid sequences stays the same, instead of increasing with length, interesting.

## But There's a Twist!
What happens if we add another `required` item to the run?

In [15]:
seq = [2,3,1,1,1,4]
series_of_runs(seq)

len(run) = 1: [[2]]  len(new_subs) = 1
len(run) = 2: [[2, 3], [3]]  len(new_subs) = 2
len(run) = 3: [[2, 3, 1], [3, 1], [1]]  len(new_subs) = 3
len(run) = 4: [[2, 3, 1, 1], [3, 1, 1], [1, 1], [1]]  len(new_subs) = 4
len(run) = 5: [[2, 3, 1, 1, 1], [3, 1, 1, 1], [1, 1, 1], [1, 1], [1]]  len(new_subs) = 5
len(run) = 6: [[2, 3, 1, 1, 1, 4], [3, 1, 1, 1, 4], [1, 1, 1, 4], [1, 1, 4], [1, 4], [4]]  len(new_subs) = 6


> ...  
> len(run) = 5: [[2, 3, 1, 1, 1], [3, 1, 1, 1], ~~[1, 1, 1]~~, ~~[1, 1]~~, ~~[1]~~]  len(new_subs) = ~~5~~ **2**  
> len(run) = **6**: [[2, 3, 1, 1, 1, 4], [3, 1, 1, 1, 4], [1, 1, 1, 4], [1, 1, 4], [1, 4], [4]]  len(new_subs) = **6**

The number of new valid sequences jumps back up to the number of items in the run!

## Putting It All Together
- Each time we add a `required` item to a valid run, it adds a number of new valid sub-sequences equal to the length of the run
- Each time we add an `optional` item to a valid run, it adds a number of new valid sub-sequences equal to the length of the run *before optional items were added*
- If we add a `required` item, previous `optional` items work like the rest of the run

Let's code that up:

In [5]:
def trailing_subs(run, optionals):
    return [ run[i:] + optionals for i in range(-len(run), 0) ]

def find_all_valid_subs(seq):
    valid_subs = []
    current_run = []
    optionals = []
    total = 0
    run_count = 0
    optionals_count = 0
    for i in seq:
        if is_disallowed(i):
            current_run = []
            optionals = []
            run_count = 0
            optionals_count = 0
            print(f"Disallowed: {i}")
            continue

        if is_required(i):
            # Optionals become part of the run
            current_run.extend(optionals)
            optionals = []
            run_count += optionals_count
            optionals_count = 0

            current_run.append(i)
            run_count += 1

        else: # is_optional(i)
            optionals.append(i)
            optionals_count += 1

        total += run_count
        new_subs = trailing_subs(current_run, optionals)
        print(f"{current_run = }, {optionals = }:\n  {new_subs = }\n  {len(new_subs) = }\n")
        valid_subs.extend(new_subs)

    assert total == len(valid_subs)  # Raise if we mis-counted
    return total

seq = [2,3,1,1,1,4]
print(f"{find_all_valid_subs(seq) = }")

current_run = [2], optionals = []:
  new_subs = [[2]]
  len(new_subs) = 1

current_run = [2, 3], optionals = []:
  new_subs = [[2, 3], [3]]
  len(new_subs) = 2

current_run = [2, 3], optionals = [1]:
  new_subs = [[2, 3, 1], [3, 1]]
  len(new_subs) = 2

current_run = [2, 3], optionals = [1, 1]:
  new_subs = [[2, 3, 1, 1], [3, 1, 1]]
  len(new_subs) = 2

current_run = [2, 3], optionals = [1, 1, 1]:
  new_subs = [[2, 3, 1, 1, 1], [3, 1, 1, 1]]
  len(new_subs) = 2

current_run = [2, 3, 1, 1, 1, 4], optionals = []:
  new_subs = [[2, 3, 1, 1, 1, 4], [3, 1, 1, 1, 4], [1, 1, 1, 4], [1, 1, 4], [1, 4], [4]]
  len(new_subs) = 6

find_all_valid_subs(seq) = 15


In [6]:
# Cool!  Let's get a little funkier:
seq = [9,2,3,1,9,1,1,4,9,1]
print(f"{find_all_valid_subs(seq)=}")

Disallowed: 9
current_run = [2], optionals = []:
  new_subs = [[2]]
  len(new_subs) = 1

current_run = [2, 3], optionals = []:
  new_subs = [[2, 3], [3]]
  len(new_subs) = 2

current_run = [2, 3], optionals = [1]:
  new_subs = [[2, 3, 1], [3, 1]]
  len(new_subs) = 2

Disallowed: 9
current_run = [], optionals = [1]:
  new_subs = []
  len(new_subs) = 0

current_run = [], optionals = [1, 1]:
  new_subs = []
  len(new_subs) = 0

current_run = [1, 1, 4], optionals = []:
  new_subs = [[1, 1, 4], [1, 4], [4]]
  len(new_subs) = 3

Disallowed: 9
current_run = [], optionals = [1]:
  new_subs = []
  len(new_subs) = 0

find_all_valid_subs(seq)=8


## Now How About O(n)
If all we need is the count, we can drop all the run and sub-sequence building:

In [7]:
def count_all_valid_subs(seq):
    total = 0
    run_count = 0
    optionals_count = 0
    for i in seq:
        if is_disallowed(i):
            run_count = 0
            optionals_count = 0
            continue

        if is_required(i):
            run_count += optionals_count + 1
            optionals_count = 0

        else: # is_optional(i)
            optionals_count += 1

        total += run_count

    return total

In [8]:
seq = [2,3,1,1,1,4]
print(f"{count_all_valid_subs(seq) = }")

count_all_valid_subs(seq) = 15


In [9]:
seq = [9,2,3,1,9,1,1,4,9,1]
print(f"{count_all_valid_subs(seq) = }")

count_all_valid_subs(seq) = 8
